# LEET CODE
## Problem-550: Game Play Analysis IV
Class: Medium

Source: https://leetcode.com/problems/game-play-analysis-iv

## Description
Table: Activity
| Column Name  | Type    |
|--------------|---------|
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |

- (player_id, event_date) is the primary key (combination of columns with unique values) of this table.
- This table shows the activity of players of some games.
- Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.

## To Do
Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

The result format is in the following example.

## Example

Input: 

Activity table:
| player_id | device_id | event_date | games_played |
|-----------|-----------|------------|--------------|
| 1         | 2         | 2016-03-01 | 5            |
| 1         | 2         | 2016-03-02 | 6            |
| 2         | 3         | 2017-06-25 | 1            |
| 3         | 1         | 2016-03-02 | 0            |
| 3         | 4         | 2018-07-03 | 5            |

Output: 
| fraction  |
|-----------|
| 0.33      |

Explanation: 
- Only the player with id 1 logged back in after the first day he had logged in so the answer is 1/3 = 0.33


## Importing

In [1]:
import pandas as pd
from pandasql import sqldf

## Define Schema

In [2]:
data = {
    'player_id': [1, 1, 2, 3, 3],
    'device_id': [2, 2, 3, 1, 4],
    'event_date': ['2016-03-01', '2016-03-02', '2017-06-25', '2016-03-02', '2018-07-03'],
    'games_played': [5, 6, 1, 0, 5]
}

Activity = pd.DataFrame(data)
Activity['event_date'] = pd.to_datetime(Activity['event_date'])

Activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


## Task

In [47]:
# Define the SQL query
query = """
SELECT
    ROUND(CAST(COUNT(*) AS float)/(SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction
FROM (SELECT * FROM Activity GROUP BY player_id HAVING MIN(event_date)) AS e1
JOIN Activity e2 USING(player_id)
WHERE DATE(e1.event_date, '+1 day') = DATE(e2.event_date)
"""

# Excute the query using pandasql
result = sqldf(query, env={'Activity':Activity})

# Display the result dataframe
display(result)

,fraction
0,0.33


In [49]:
# For MYSQL submission:
"""
SELECT
  ROUND(COUNT(DISTINCT player_id) / (SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction
FROM
  Activity
WHERE
  (player_id, DATE_SUB(event_date, INTERVAL 1 DAY))
  IN (
    SELECT player_id, MIN(event_date) AS first_login FROM Activity GROUP BY player_id
  )
"""